긴 문장으로 부터 다음 문장을 예측하는 모델을 만들어보자.</br>
긴 문장을 특정 사이즈로 잘라서 훈련한다.</br>

In [1]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

# make dictionary
char_set = list(set(sentence))
# character 사전을 만든다. index -> char
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
# 긴 문장을 자를 크기를 설정한다.
sequence_length = 10
learning_rate = 0.1

# data setting
x_data = []
y_data = []

# 가져올 수 있는 청크의 개수만큼 루프를 돈다.
chunk_count = len(sentence) - sequence_length
for i in range(0, chunk_count):
    x_str = sentence[i:i + sequence_length]             # 예를들어 "if you wan"가 입력이면
    y_str = sentence[i + 1: i + sequence_length + 1]    # 예를들어 "f you want"가 정답이 된다.
    # print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

# np.eye 함수를 사용해서 원-핫 벡터로 만든다. 
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)


# 좀 더 크고 복잡한 모델을 만들기 위해서 RNN을 스택으로 쌓고 Fully Connected로 연결한다.
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(
    dic_size, 
    hidden_size,
    2   # layer를 2로 설정해서 RRN을 2 스택으로 한다.
)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()   # gradient를 초기화한다.
    outputs = net(X)        # 모델에 입력을 넣는다.
    # 모델의 output과 label을 비교해서 loss를 구한다.
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()    # optimizer에 넣었던 네트워크의 파라미터를 업데이트한다.

    # 3번째 차원의 값들중 가장 큰 값의 인덱스를 가져온다. 
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]     # 맨 마지막 결과를 가져온다.

    print(predict_str)


b'wtt wttwt'wwwwt ''wt'w'w'twwtw'''wtww'wt'ttwttt'w'tww'ttww'wwtwtt'''wtww'twt''tw'''w'w 'wwt'wt'w''w'wwwe'ttwwt'wtw'ttw'wwwwwwb''w'ww'wtwwtwt''ttww'wwww''t''wwtttt' ''wt'ww'ww'ww
                                                                                                                                                                                   
totootoottottotoottottoottotttotttttottotttttotottootooototottootoottottottottttotoottottottoottttotottootttottottootoottottoooootoottotttoottottootttoottotttttototottootootottott
 kb l.e k..e...''..'h.'e.......'...h.e..h....e.e...n..he.''...h...n..eh.h..h..h....h...e.ee..h..........ee.....e..hh..e..e'...e....e'.....'...eh.'e....h..e'....'......e.........h.
aalua la llalllullllallaulullulallullalllllllallallaallallallllllluallalllllllallllulllallullullllllullllalllllllllallulllalllallllaalllllullllallullllallualllllllllllllllllllllal
e le ha haaenaaeneanelaaenataaae aeaaeneanlaaenaeaaaeaa leelana naatne ne aalnenaeaaeaaenaanneennane